In [1]:
import pandas as pd
import os
import re
import subprocess
import contextlib
import math
import mmap
import requests
import lxml.html as lh

In [2]:
#single cell DE data as binary matrix 
sc=pd.read_csv("../data/scRNAseq_Cengen2019/110119_sc_neuron_class_vs_all_DE_NB_binary.csv",sep=',')

#import brain atlas data
ba=pd.read_csv("../data/BrainAtlas/091719_Binary Brain atlas V16 new homeobox data 082619.csv",sep=",")

#all C. elegans TFs
All_eleg_TFs=pd.read_csv("../data/motif_information/Table_1_Ce_TF_list_Wormbook_2013 Reinke et al.csv",sep=',')

cv2=pd.read_csv("../data/motif_information/TF_Information_all_motifs_plus_cisbp_v2.0.txt",sep='\t')

In [3]:
#get a list of TFs in sc and ba data.
sc_genes=set(list(sc["gene_name"].values)[2:])
ba_genes=set(list(ba.columns.values)[4:])
all_genes=sc_genes | ba_genes
all_TFs=set(list(All_eleg_TFs["Gene Public Name"].values)+list(All_eleg_TFs["Sequence Name (Gene)"].values))
n_TFs=set(all_TFs & all_genes)
print('neuronal TFs:',len(n_TFs))

neuronal TFs: 389


In [4]:
#get TF_ID from cisbp for each TF
d=pd.DataFrame(list(n_TFs),columns=['TF_Name'])
m=d.merge(cv2[['TF_Name','TF_ID']], on='TF_Name').drop_duplicates()
print('neuronal TF with motif:', len(m))
m.head(2)

neuronal TF with motif: 319


TF_Name         TF_ID
0    ceh-31  T224616_2.00
447   gei-3  T333266_2.00

In [9]:
def getHTML_data_I(TF_ID):
    url='http://cisbp.ccbr.utoronto.ca/TFreport.php?searchTF=%s' %TF_ID
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//tr')
    I_motif_elements=[row for row in tr_elements if len(row)==8]
    return I_motif_elements

In [10]:
def getHTML_data_D(TF_ID):
    url='http://cisbp.ccbr.utoronto.ca/TFreport.php?searchTF=%s' %TF_ID
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//tr')
    motif_elements=[row for row in tr_elements if len(row)==6]
    return motif_elements

In [11]:
def extract_info_D(motif_elements):
    cols=['TF_Name','evidence','TF_ID','protein', 'motif_ID', 'species', 'M_source', 'M_author', 'M_year','SR_score', 'DB_score']
    d=pd.DataFrame(columns=cols)
    exclusion_rule='None'
    t =motif_elements[2]
    name=t.text_content()
    n=name.split()
    #print(n)
    motif_ID=n[0]
    
    #get similar TF info
    print('list similar in D')
    list_similar=''
    for t in motif_elements[:]:
        name=t.text_content()
        n=name.split()
        #print(n)
        species="%s %s" %(n[1],n[2])
        #print(species)
        if species=="Caenorhabditis elegans":
            SR_score=n[5]
            protein=n[0]
            x='%s,%s; ' %(protein,SR_score)
            list_similar=list_similar+x
            if float(SR_score)>SR_max:
                exclusion_rule=2
    return motif_ID,list_similar,exclusion_rule


In [12]:

def extract_info_I(motif_elements):
    cols=['TF_Name','evidence','TF_ID','protein', 'motif_ID', 'species', 'M_source', 'M_author', 'M_year','SR_score', 'DB_score']
    d=pd.DataFrame(columns=cols)
    if len(motif_elements)>0:
        t =motif_elements[0]
        name=t.text_content()
        n=name.split()
        #print(n)
        motif_ID=n[1]
        protein=n[0]
        SR=n[-2]
        species='%s %s' %(n[2],n[3])
        TF_Status='I'
        MSource_type, MSource_Author_Year=[n[6],'%s %s %s' %(n[7],n[8],n[9])]
        
        exclusion_rule='None'
        if species=='Caenorhabditis elegans':
            exclusion_rule=1
        
        return motif_ID,protein,SR, species,TF_Status,MSource_type, MSource_Author_Year,exclusion_rule

    else:return 'NA','NA','NA','NA','NA','NA','NA',0

In [28]:
#for each TF, get cisbp page data
#largest similary score allowed to include motif (set by allowing unc-42 which has SR score .807 to ceh-17 )

#Rules for filtering. (Exclusion rules)
#0. No direct experiments, no TFs with similar DNA binding domains (no Motif available)
#1. If best inferred motif is C. elegans TF, exclude 
#2. if C. elegans TF with similar DBD has SR score > .808, exclude (value that allows inclusion of unc-42 motif)
#3. Motif from Hobert lab in vivo promoter basing used instead (unc-3, che-1)
#4. Duplicate Inferred motif use for more than one C. elegans TF. (sox-2 and sox-3)
#5. Inferred motif SR score too close to SR score of other C. elegans TFs. (Unc-39) -added exclusion code to these manually.
#6. known repressor.
#None. do not exclude

SR_max=.808
df=pd.DataFrame()
for TF_ID, TF_Name in m[['TF_ID','TF_Name']].values[:]:
    exclusion_rule='None'
    print(TF_Name, TF_ID)
    motif_elements=getHTML_data_D(TF_ID)
    motif_ID, similar,exclusion_rule=extract_info_D(motif_elements)
    TF_Status='D'
    protein=TF_Name
    SR='Direct'
    species='Caenorhabditis elegans'
    #get study type, author, year from cisbp table
    
 
    if motif_ID=='No':
        motif_elements=getHTML_data_I(TF_ID)
        motif_ID, protein, SR,species, TF_Status,MSource_type,MSource_Author_Year,exclusion_rule=extract_info_I(motif_elements)
        #if similar list contains SR score > SR_max, set exclusion rule to 2. 
        for s in similar.split(';'):
            x=s.split(',')
            if len(x)>1:
                if float(x[1])>SR_max:
                    exclusion_rule=2
    else:
        try:
            l=list(cv2.loc[(cv2.TF_ID==TF_ID) & (cv2.Motif_ID==motif_ID)][['MSource_Type', 'MSource_Author','MSource_Year']].values[0])
            MSource_type, MSource_Author_Year=[l[0],'%s et al. %s' %(l[1],l[2])]
        except:
            print(TF_Name, TF_ID, 'motif error')
            MSource_type, MSource_Author_Year=['NA','NA']
    
    result=[TF_Name, TF_Status,TF_ID, motif_ID, protein,species, SR,similar,  MSource_type, MSource_Author_Year,exclusion_rule]
    headers='TF_Name,TF_Status,TF_ID,motif_ID,protein,species,SR,similar,MSource_type,MSource_Author_Year,exclusion_rule'.split(',')
    dictionary = dict(zip(headers, result))
    df=df.append(dictionary,ignore_index=True)
    print(result,'\n')

hlh-16 T046284_2.00
list similar in D
['hlh-16', 'D', 'T046284_2.00', 'M00589_2.00', 'hlh-16', 'Caenorhabditis elegans', 'Direct', '', 'PBM', 'Narasimhan et al. 2015', 'None'] 

nhr-60 T306174_2.00
list similar in D
['nhr-60', 'I', 'T306174_2.00', 'M00699_2.00', 'nhr-168', 'Caenorhabditis elegans', '0.976', 'nhr-168,0.976; nhr-116,0.929; nhr-141,0.822; nhr-129,0.818; ', 'PBM', 'Narasimhan et al.(2015)', 2] 

isw-1 T282144_2.00
list similar in D
['isw-1', 'NA', 'T282144_2.00', 'NA', 'NA', 'NA', 'NA', '', 'NA', 'NA', 0] 

let-607 T069400_2.00
list similar in D
['let-607', 'NA', 'T069400_2.00', 'NA', 'NA', 'NA', 'NA', '', 'NA', 'NA', 0] 

ceh-10 T224602_2.00
list similar in D
['ceh-10', 'I', 'T224602_2.00', 'M00287_2.00', 'VSX2', 'Homo sapiens', '0.801', 'unc-42,0.773; unc-4,0.760; ceh-17,0.749; alr-1,0.746; ceh-8,0.734; lim-6,0.690; dsc-1,0.672; ttx-3,0.664; zfh-2,0.661; ceh-62,0.658; vab-7,0.653; ceh-16,0.648; ceh-14,0.646; vab-15,0.638; ceh-1,0.628; lim-4,0.608; ', 'PBM', 'Barrera et a

list similar in D
['egl-13', 'D', 'T333265_2.00', 'M00710_2.00', 'egl-13', 'Caenorhabditis elegans', 'Direct', 'sox-3,0.588; sox-2,0.555; sox-4,0.544; sem-2,0.543; ', 'PBM', 'Narasimhan et al. 2015', 'None'] 

nhr-231 T306298_2.00
list similar in D
['nhr-231', 'I', 'T306298_2.00', 'M00700_2.00', 'nhr-232', 'Caenorhabditis elegans', '0.881', 'nhr-170,0.979; nhr-54,0.961; nhr-232,0.881; nhr-142,0.862; nhr-213,0.850; nhr-178,0.814; nhr-102,0.765; ', 'PBM', 'Narasimhan et al.(2015)', 2] 

nhr-17 T306141_2.00
list similar in D
['nhr-17', 'I', 'T306141_2.00', 'M00686_2.00', 'nhr-1', 'Caenorhabditis elegans', '0.826', 'nhr-10,0.886; nhr-101,0.846; nhr-47,0.829; nhr-1,0.826; nhr-40,0.799; nhr-68,0.780; ', 'PBM', 'Narasimhan et al.(2015)', 2] 

nhr-277 T306384_2.00
list similar in D
['nhr-277', 'NA', 'T306384_2.00', 'NA', 'NA', 'NA', 'NA', '', 'NA', 'NA', 0] 

let-526 T015802_2.00
list similar in D
['let-526', 'NA', 'T015802_2.00', 'NA', 'NA', 'NA', 'NA', '', 'NA', 'NA', 0] 

sea-2 T131571_2.00

list similar in D
['nhr-58', 'NA', 'T306172_2.00', 'NA', 'NA', 'NA', 'NA', '', 'NA', 'NA', 0] 

pha-2 T224638_2.00
list similar in D
['pha-2', 'D', 'T224638_2.00', 'M02184_2.00', 'pha-2', 'Caenorhabditis elegans', 'Direct', '', 'PBM', 'Weirauch et al. 2014', 'None'] 

ttx-3 T224642_2.00
list similar in D
['ttx-3', 'I', 'T224642_2.00', 'M02085_2.00', 'LHX2', 'Homo sapiens', '0.844', 'ceh-14,0.725; ceh-8,0.725; alr-1,0.707; zfh-2,0.702; ceh-17,0.702; unc-42,0.697; lim-4,0.697; unc-4,0.691; lim-6,0.680; ceh-16,0.666; ceh-10,0.664; ceh-2,0.647; vab-15,0.618; dsc-1,0.610; ceh-5,0.608; lin-11,0.606; mec-3,0.605; ceh-62,0.604; ', 'PBM', 'Weirauch et al.(2014)', 'None'] 

tbx-2 T340933_2.00
list similar in D
['tbx-2', 'I', 'T340933_2.00', 'M00830_2.00', 'Tbx2', 'Mus musculus', '0.790', '', 'PBM', 'Weirauch et al.(2013)', 'None'] 

daf-16 T188799_2.00
list similar in D
['daf-16', 'D', 'T188799_2.00', 'M01989_2.00', 'daf-16', 'Caenorhabditis elegans', 'Direct', '', 'PBM', 'Weirauch et al. 2014',

list similar in D
['eor-1', 'D', 'T131555_2.00', 'M01865_2.00', 'eor-1', 'Caenorhabditis elegans', 'Direct', '', 'PBM', 'Weirauch et al. 2014', 'None'] 

unc-62 T224647_2.00
list similar in D
['unc-62', 'D', 'T224647_2.00', 'M02185_2.00', 'unc-62', 'Caenorhabditis elegans', 'Direct', '', 'PBM', 'Weirauch et al. 2014', 'None'] 

mef-2 T256826_2.00
list similar in D
['mef-2', 'I', 'T256826_2.00', 'M02745_2.00', 'MEF2C', 'Homo sapiens', '0.958', '', 'SELEX', 'Jolma et al.(2010)', 'None'] 

nhr-189 T306353_2.00
list similar in D
['nhr-189', 'NA', 'T306353_2.00', 'NA', 'NA', 'NA', 'NA', 'nhr-188,0.929; ', 'NA', 'NA', 2] 

maf-1 T069421_2.00
list similar in D
['maf-1', 'I', 'T069421_2.00', 'M01814_2.00', 'Mafg', 'Mus musculus', '0.814', '', 'PBM', 'Weirauch et al.(2014)', 'None'] 

mls-2 T224635_2.00
list similar in D
['mls-2', 'I', 'T224635_2.00', 'M03233_2.00', 'HMX3', 'Homo sapiens', '0.832', 'ceh-9,0.683; lin-39,0.659; ceh-2,0.632; vab-15,0.632; ceh-63,0.626; tab-1,0.617; ceh-12,0.610; c

list similar in D
['sex-1', 'D', 'T306252_2.00', 'M02416_2.00', 'sex-1', 'Caenorhabditis elegans', 'Direct', 'nhr-23,0.966; nhr-41,0.959; unc-55,0.951; nhr-85,0.930; nhr-91,0.876; nhr-2,0.832; ', 'PBM', 'Weirauch et al. 2014', 2] 

nhr-180 T306341_2.00
list similar in D
['nhr-180', 'I', 'T306341_2.00', 'M02419_2.00', 'nhr-177', 'Caenorhabditis elegans', '0.779', 'nhr-95,0.954; nhr-228,0.936; nhr-115,0.857; nhr-125,0.855; nhr-225,0.835; WBGene00020591,0.812; nhr-177,0.779; nhr-226,0.753; ', 'PBM', 'Weirauch et al.(2014)', 2] 

fax-1 T306124_2.00
list similar in D
['fax-1', 'I', 'T306124_2.00', 'M06432_2.00', 'Hr51', 'Drosophila melanogaster', '0.980', '', 'B1H', 'Zhu et al.(2011)', 'None'] 

nhr-255 T306308_2.00
list similar in D
['nhr-255', 'NA', 'T306308_2.00', 'NA', 'NA', 'NA', 'NA', '', 'NA', 'NA', 0] 

nhr-254 T306389_2.00
list similar in D
['nhr-254', 'NA', 'T306389_2.00', 'NA', 'NA', 'NA', 'NA', 'nhr-253,0.975; nhr-252,0.922; WBGene00022636,0.917; nhr-167,0.895; ', 'NA', 'NA', 2]

list similar in D
['aptf-1', 'NA', 'T011139_2.00', 'NA', 'NA', 'NA', 'NA', '', 'NA', 'NA', 0] 

nhr-52 T306166_2.00
list similar in D
['nhr-52', 'NA', 'T306166_2.00', 'NA', 'NA', 'NA', 'NA', '', 'NA', 'NA', 0] 

ceh-28 T224614_2.00
list similar in D
ceh-28 T224614_2.00 motif error
['ceh-28', 'D', 'T224614_2.00', 'Name', 'ceh-28', 'Caenorhabditis elegans', 'Direct', 'ceh-27,0.817; ceh-24,0.797; ceh-22,0.709; ', 'NA', 'NA', 2] 

nhr-1 T306125_2.00
list similar in D
['nhr-1', 'D', 'T306125_2.00', 'M00686_2.00', 'nhr-1', 'Caenorhabditis elegans', 'Direct', 'nhr-10,0.987; nhr-49,0.956; nhr-101,0.950; nhr-68,0.929; nhr-47,0.905; nhr-120,0.890; nhr-35,0.882; nhr-173,0.848; nhr-40,0.835; nhr-17,0.826; ', 'PBM', 'Narasimhan et al. 2015', 2] 

ceh-39 T167389_2.00
list similar in D
['ceh-39', 'NA', 'T167389_2.00', 'NA', 'NA', 'NA', 'NA', 'ceh-21,0.726; ', 'NA', 'NA', 0] 

sem-2 T333273_2.00
list similar in D
['sem-2', 'D', 'T333273_2.00', 'M02488_2.00', 'sem-2', 'Caenorhabditis elegans', 'Direct'

list similar in D
['ceh-36', 'D', 'T224620_2.00', 'M00673_2.00', 'ceh-36', 'Caenorhabditis elegans', 'Direct', 'ceh-37,0.774; ceh-53,0.755; ttx-1,0.755; unc-30,0.710; ceh-45,0.709; ', 'PBM', 'Narasimhan et al. 2015', 'None'] 

hlh-2 T046273_2.00
list similar in D
['hlh-2', 'NA', 'T046273_2.00', 'NA', 'NA', 'NA', 'NA', '', 'NA', 'NA', 0] 

nhr-37 T306346_2.00
list similar in D
['nhr-37', 'I', 'T306346_2.00', 'M02417_2.00', 'nhr-145', 'Caenorhabditis elegans', '0.844', 'nhr-145,0.844; ', 'PBM', 'Weirauch et al.(2014)', 2] 

nhr-20 T306144_2.00
list similar in D
['nhr-20', 'NA', 'T306144_2.00', 'NA', 'NA', 'NA', 'NA', '', 'NA', 'NA', 0] 

unc-86 T238867_2.00
list similar in D
['unc-86', 'D', 'T238867_2.00', 'M02244_2.00', 'unc-86', 'Caenorhabditis elegans', 'Direct', '', 'PBM', 'Weirauch et al. 2014', 'None'] 

ets-5 T177602_2.00
list similar in D
['ets-5', 'I', 'T177602_2.00', 'M03004_2.00', 'FEV', 'Homo sapiens', '0.995', 'ast-1,0.993; lin-1,0.945; elf-1,0.922; ets-7,0.902; F19F10.1,0.8

list similar in D
['unc-42', 'I', 'T224646_2.00', 'M03874_2.00', 'CG32532', 'Drosophila melanogaster', '0.836', 'ceh-17,0.807; alr-1,0.802; unc-4,0.789; ceh-8,0.784; ceh-10,0.773; ceh-14,0.704; ttx-3,0.697; ceh-16,0.691; vab-15,0.683; lim-6,0.677; ceh-62,0.673; lim-4,0.666; zfh-2,0.663; dsc-1,0.658; ceh-1,0.657; mec-3,0.650; lin-11,0.646; ceh-2,0.642; tab-1,0.642; ceh-12,0.635; ceh-5,0.623; vab-7,0.612; ceh-43,0.611; ', 'SELEX', 'Nitta et al.(2015)', 'None'] 

pbrm-1 T333276_2.00
list similar in D
['pbrm-1', 'NA', 'T333276_2.00', 'NA', 'NA', 'NA', 'NA', '', 'NA', 'NA', 0] 

aha-1 T046268_2.00
list similar in D
['aha-1', 'NA', 'T046268_2.00', 'NA', 'NA', 'NA', 'NA', '', 'NA', 'NA', 0] 

flh-2 T183382_2.00
list similar in D
['flh-2', 'NA', 'T183382_2.00', 'NA', 'NA', 'NA', 'NA', '', 'NA', 'NA', 0] 

ztf-6 T131601_2.00
list similar in D
['ztf-6', 'D', 'T131601_2.00', 'M00631_2.00', 'ztf-6', 'Caenorhabditis elegans', 'Direct', '', 'PBM', 'Narasimhan et al. 2015', 'None'] 

vab-7 T224648_2.

In [29]:
df.to_csv("../data/motif_information/curated_motifs_110719.csv",sep=',')

In [128]:
#write script to add exclusion rules 4 and 5 (if rule =='None', check for 4 and 5,6)
#3. Motif from Hobert lab in vivo promoter basing used instead (unc-3, che-1, hlh-4)
#4. Duplicate Inferred motif use for more than one C. elegans TF. (sox-2 and sox-3)
#5. Inferred motif SR score too close to SR score of other C. elegans TFs. (Unc-39) -added exclusion code to these manually.
#6. known repressor.

df=pd.read_csv("../data/motif_information/curated_motifs_110719.csv",sep=',')
headers='TF_Name,TF_Status,TF_ID,motif_ID,protein,species,SR,similar,MSource_type,MSource_Author_Year,exclusion_rule'.split(',')
df=df[headers]


#use previous motif info to add OH motifs and motif names for a few TFs that didn't propertly fetch motif_ID from webscrape
df2=pd.read_csv("../data/motif_information/curated_motifs_110319.csv",sep=',')
df2=df2[headers]



def add_rules(df):
    exclude_TFs=['unc-3','che-1','hlh-4']
    repressors=['bnc-1','mab-9','cog-1']
    
    SR_cutoff_rule5=.03
    exclusion_rules=[]
    mlist=[]
    for TF_Name,motif_ID, SR, similar, exclusion_rule in df[['TF_Name','motif_ID','SR','similar','exclusion_rule']].values:
        e=exclusion_rule
        mname=motif_ID

        #if motif ID is found >1, set exclusion rule to 4.
        if motif_ID!='Name':
            if len(df.loc[df['motif_ID']==motif_ID])>1:
                print(TF_Name,motif_ID,'exluded with rule 4')
                e=4
            #if SR score too score to other C. elegans TF, exlude (rule 5)
            #first similar SR score is highest. compare this one
        if motif_ID=='Name':
            mname=df2.loc[df2['TF_Name']==TF_Name]["motif_ID"].values[0]
            print(TF_Name, motif_ID, mname)

        elif type(similar)!=float:
            if SR!='Direct':
                similar_SR=float(similar.split(';')[0].split(',')[1])
                SR=float(SR)
                if abs(SR-similar_SR)/SR<(SR_cutoff_rule5):
                    print(TF_Name,SR, similar_SR,'exluded with rule 5')
                    e=5
        #rule 3 and repressors  (exlude motifs that we have better in vivo derived motifs for from prom bashing)
        elif TF_Name in exclude_TFs:
            e=3
            print(TF_Name, 'excluded with rule 3')
        elif TF_Name in repressors:
            e=6
            print(TF_Name,'excluded with rule 6')
        exclusion_rules.append(e)
        mlist.append(mname)

    print(len(exclusion_rules))
    df['exclusion_rule']=exclusion_rules
    df['motif_ID']=mlist
    return df

df=add_rules(df)
#run again to make sure new motif names aren't redudant (redo rule 4 check)
df=add_rules(df)

#add OH motifs to list
add_these_motifs=['OH2018_HLH-4', 'OH2011_UNC-3', 'OH2007_CEH-10_TTX-3','OH2004_CHE-1', 'M03312_2.00']
df2=pd.read_csv("../data/motif_information/curated_motifs_110319.csv",sep=',')

OH_motif_info=df2.loc[df2['motif_ID'].isin(add_these_motifs)]
df=df.append(OH_motif_info)


#add neuronal TF names that didn't have TF_IDs or motif info in cisbp database
missing_n_TFs=list((set(n_TFs)-set(df["TF_Name"].values)))
d=pd.DataFrame(missing_n_TFs,columns=['TF_Name'])
d['exclusion_rule']=0
df=df.append(d)
df=df[headers]
df.to_csv("../data/motif_information/curated_motifs_110719b.csv",sep=',')

nhr-60 0.976 0.976 exluded with rule 5
dpl-1 Name M09525_2.00
nhr-62 M00690_2.00 exluded with rule 4
sox-4 0.577 0.566 exluded with rule 5
nhr-47 M00686_2.00 exluded with rule 4
nhr-47 0.905 0.912 exluded with rule 5
lin-15B Name M09654_2.0
nhr-120 M02413_2.00 exluded with rule 4
ceh-31 M03810_2.00 exluded with rule 4
ceh-20 0.853 0.849 exluded with rule 5
nhr-231 M00700_2.00 exluded with rule 4
nhr-17 M00686_2.00 exluded with rule 4
ceh-30 M03810_2.00 exluded with rule 4
egl-5 Name M09581_2.00
nhr-3 M00687_2.00 exluded with rule 4
che-1 excluded with rule 3
nhr-88 M00687_2.00 exluded with rule 4
hlh-4 excluded with rule 3
nhr-133 M00701_2.00 exluded with rule 4
mec-3 M00405_2.00 exluded with rule 4
mec-3 0.846 0.846 exluded with rule 5
cog-1 excluded with rule 6
nhr-178 M00703_2.00 exluded with rule 4
nhr-45 M00700_2.00 exluded with rule 4
unc-55 0.981 0.968 exluded with rule 5
egl-27 Name M08205_2.00
sox-3 M03519_2.00 exluded with rule 4
nhr-98 M00701_2.00 exluded with rule 4
nhr-21 

In [ ]:
#add TF that had no TFID in cisbp. -assign exlusion rule 0 (no data).

In [107]:
#check these 134 motifs for targetortho results. 
tfiles=os.listdir('../data/TargetOrtho_data/')
motif_dic={}
for m,TF,e in df[['motif_ID','TF_Name','exclusion_rule']].loc[df['exclusion_rule']=='None'].values[:]:
    #print(m,TF)
    found=False
    for t in tfiles:
        if m in t:
            if TF=='sox-3':
                print(m,t)
            found=True
    if found==False:
        motif_dic[m]=TF
        #print(m)
motif_dic

{}

In [354]:
#for each motif, write meme format motif file. 
import math
psuedo_count= .000000001 
bfs=[.25,.25,.25,.25]
A=bfs[0]
C=bfs[1]
G=bfs[2]
T=bfs[3]

for M in motif_dic.keys():
    
    print(motif_dic[M],M)
    TF_Name=motif_dic[M]
    Motif_Name=M
    outfile=open("../data/motif_information/cisbp_v2_46_new_motifs/%s_%s_meme.txt" %(TF_Name,Motif_Name),'w')

    p="""MEME version 4\nALPHABET= ACGT\nstrands: + -\nBackground letter frequencies (from uniform background):\nA 0.25000 C 0.25000 G 0.25000 T 0.25000\nMOTIF %s\nlog-odds matrix: alength=4 width = n\n""" %(Motif_Name)
    outfile.write(p)

    f="../data/motif_information/pwms_entire_Cisbp_v2_database/%s.txt" %Motif_Name
    print(TF_Name,Motif_Name,f)
    f=open(f).readlines()

    for line in f:
        l=line.split('\t')
        if l[0]!='Pos':
            vals=[float(i) for i in l[1:]]
            #print(vals)
            l=[int(math.log(float(i+ psuedo_count)/A,2)*100) for i in vals]
            p="%s\t%s\t%s\t%s\n" %(l[0],l[1],l[2],l[3])
            print(p)
            outfile.write(p)
    outfile.close()

ceh-13 M00450_2.00
ceh-13 M00450_2.00 ../data/motif_information/pwms_entire_Cisbp_v2_database/M00450_2.00.txt
-34	5	-20	38

-57	-78	-130	123

148	-855	-20	-161

194	-976	-520	-314

-132	-344	-152	166

-165	-90	24	97

124	-286	30	-198

-25	36	24	-54

-63	45	-27	21

ceh-32 M00483_2.00
ceh-32 M00483_2.00 ../data/motif_information/pwms_entire_Cisbp_v2_database/M00483_2.00.txt
-1	-4	8	-2

-41	-2	-29	53

-27	-78	80	-24

171	-115	-289	-280

-98	-54	-132	126

110	43	-252	-164

-76	74	-47	2

tab-1 M00516_2.00
tab-1 M00516_2.00 ../data/motif_information/pwms_entire_Cisbp_v2_database/M00516_2.00.txt
9	-12	-28	25

-110	-159	-298	162

182	-983	-136	-394

193	-395	-941	-318

-211	-35	-160	140

-125	-11	41	40

38	-169	81	-69

-39	12	43	-32

-55	35	-51	42

vab-15 M00555_2.00
vab-15 M00555_2.00 ../data/motif_information/pwms_entire_Cisbp_v2_database/M00555_2.00.txt
-128	-183	-293	166

166	-244	-175	-150

192	-1019	-355	-320

-285	-357	-227	183

-168	-187	-138	160

48	-272	102	-125

-21	12	38	-43

-23	1

In [355]:
#transfer meme files to server
#write Targetortho commands for sh file to run on server.
for M in motif_dic.keys():   
    TF_Name=motif_dic[M]
    Motif_Name=M

    p="python targetortho.py -f data/cisbp_v2_46_new_motifs/%s_%s_meme.txt -p 0.0001 -j %s_%s_;" %(TF_Name, Motif_Name,TF_Name, Motif_Name)
    print(p) 

python targetortho.py -f data/cisbp_v2_46_new_motifs/ceh-13_M00450_2.00_meme.txt -p 0.0001 -j ceh-13_M00450_2.00_;
python targetortho.py -f data/cisbp_v2_46_new_motifs/ceh-32_M00483_2.00_meme.txt -p 0.0001 -j ceh-32_M00483_2.00_;
python targetortho.py -f data/cisbp_v2_46_new_motifs/tab-1_M00516_2.00_meme.txt -p 0.0001 -j tab-1_M00516_2.00_;
python targetortho.py -f data/cisbp_v2_46_new_motifs/vab-15_M00555_2.00_meme.txt -p 0.0001 -j vab-15_M00555_2.00_;
python targetortho.py -f data/cisbp_v2_46_new_motifs/atf-7_M00595_2.00_meme.txt -p 0.0001 -j atf-7_M00595_2.00_;
python targetortho.py -f data/cisbp_v2_46_new_motifs/zip-3_M00601_2.00_meme.txt -p 0.0001 -j zip-3_M00601_2.00_;
python targetortho.py -f data/cisbp_v2_46_new_motifs/crh-2_M00603_2.00_meme.txt -p 0.0001 -j crh-2_M00603_2.00_;
python targetortho.py -f data/cisbp_v2_46_new_motifs/cebp-1_M00604_2.00_meme.txt -p 0.0001 -j cebp-1_M00604_2.00_;
python targetortho.py -f data/cisbp_v2_46_new_motifs/lin-29_M00613_2.00_meme.txt -p 0.00